# Going Bananas

In today's guided project, we'll work with [Instacart](https://www.kaggle.com/c/instacart-market-basket-analysis) dataset on Kaggle. This is a large dataset, so our project will go smoother if we all have some baseline knowledge about the dataset before we start.

# Activity

In small groups, download and explore the dataset. (The code for downloading the data and reading the four `CSV` files we're interested in is below.) Then complete the following steps:

1. Read the [description of the data on Kaggle](https://www.kaggle.com/c/instacart-market-basket-analysis/data) so that you're clear on what information is in the four `csv` files we'll examine in class. 
2. Use the `head` method and `shape` attribute to see how much data we're dealing with what data is in each DataFrame. 

Next, as a group, come up with answers to the following questions:

1. What information is contained in the column `orders['eval_set']`?
2. The first row of `orders['order_id']` is `2539329`. Where can we find the items that were included in that order?
3. The first row of `order_products_prior['product_id']` is `33120`. What is the name of that product?

We'll have a sli.do poll at the start of class to go over your answers. 

In [ ]:
import pandas as pd
import requests
import tarfile

# Download data from AWS
def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

# Extract Files
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

# Load files into DataFrames
orders = pd.read_csv('instacart_2017_05_01/orders.csv')
order_products_train = pd.read_csv('instacart_2017_05_01/order_products__train.csv')
order_products_prior = pd.read_csv('instacart_2017_05_01/order_products__prior.csv')
products = pd.read_csv('instacart_2017_05_01/products.csv')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


# Answers to Pre-class Activity

In [ ]:
orders.sort_values(['user_id', 'order_number']).head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [ ]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

In [ ]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0


In [ ]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [ ]:
mask = order_products_prior['order_id'] == 2539329
order_products_prior[mask]

,order_id,product_id,add_to_cart_order,reordered
24076664,2539329,196,1,0
24076665,2539329,14084,2,0
24076666,2539329,12427,3,0
24076667,2539329,26088,4,0
24076668,2539329,26405,5,0


In [ ]:
products[products.product_id == 33120]

,product_id,product_name,aisle_id,department_id
33119,33120,Organic Egg Whites,86,16


# EDA

What are all these DataFrames?

In [ ]:
from IPython.display import display

frames = [orders, order_products_prior, order_products_train, products]

for frame in frames:
  print(frame.shape)
  display(frame.head())
  print()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0



(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


We have four DataFrames. Can we consolidate?

In [ ]:
order_products_prior.shape[1] == order_products_train.shape[1]

True

In [ ]:
order_products_prior.columns == order_products_train.columns

array([ True,  True,  True,  True])

In [ ]:
order_products = pd.concat([order_products_prior, order_products_train])

In [ ]:
order_products.shape

(33819106, 4)

In [ ]:
products.shape

(49688, 4)

In [ ]:
order_products = pd.merge(order_products, products, on='product_id', how='left')

In [ ]:
order_products.shape

(33819106, 7)

In [ ]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,26,33120,5,0,Organic Egg Whites,86,16
2,120,33120,13,0,Organic Egg Whites,86,16
3,327,33120,5,1,Organic Egg Whites,86,16
4,390,33120,28,1,Organic Egg Whites,86,16


Now we have two DataFrames: `orders` and `order_products`.

# Define Our Problem

- Will a customer order one particular product?
- Which product? The most frequently ordered product?
- What is the most frequently ordered product?

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,2,33120,1,1,Organic Egg Whites,86,16
1,2,28985,2,1,Michigan Organic Kale,83,4
2,2,9327,3,0,Garlic Powder,104,13
3,2,45918,4,1,Coconut Butter,19,13
4,2,30035,5,0,Natural Sweetener,17,13


In [ ]:
order_products['product_name'].value_counts().head()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Name: product_name, dtype: int64

# What are we going to predict?

Will a customer order bananas (either a single `'Banana'` or a `'Bag of Organic Bananas'`?

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [ ]:
orders.eval_set.value_counts()

prior    3214874
train     131209
test       75000
Name: eval_set, dtype: int64

# Make our training set

## Step 1: Create our feature matrix from `orders`

In [ ]:
mask = orders.eval_set == 'train'
X = orders[mask]

In [ ]:
X.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


## Step 2: Create our targets

Identify when a banana is ordered in `order_products`

In [ ]:
banana_ids = [13176, 24852, 39276]

mask = order_products.product_id.isin(banana_ids)
order_products[mask].head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
30,5,13176,1,1,Bag of Organic Bananas,24,4
77,10,24852,1,1,Banana,24,4
180,20,24852,6,0,Banana,24,4
190,22,24852,3,1,Banana,24,4
234,26,24852,2,1,Banana,24,4


In [ ]:
order_products['is_banana'] = order_products.product_id.isin(banana_ids)

In [ ]:
order_products[mask].head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,is_banana
30,5,13176,1,1,Bag of Organic Bananas,24,4,True
77,10,24852,1,1,Banana,24,4,True
180,20,24852,6,0,Banana,24,4,True
190,22,24852,3,1,Banana,24,4,True
234,26,24852,2,1,Banana,24,4,True


Identify the `order_id`s that contain bananas

In [ ]:
X.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [ ]:
banana_order_ids = order_products[order_products['is_banana']].order_id.unique()

In [ ]:
banana_order_ids.shape

(890509,)

In [ ]:
X['order_banana'] = X.order_id.isin(banana_order_ids)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
X.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_banana
10,1187899,1,train,11,4,8,14.0,False
25,1492625,2,train,15,1,11,30.0,True
49,2196797,5,train,5,0,11,6.0,False
74,525192,7,train,21,2,11,6.0,False
78,880375,8,train,4,1,14,10.0,False


Create our feature matrix and targets.

In [ ]:
y = X['order_banana']
X = X[['order_dow', 'order_hour_of_day', 'days_since_prior_order']]

In [ ]:
X.head()

,order_dow,order_hour_of_day,days_since_prior_order
10,4,8,14.0
25,1,11,30.0
49,0,11,6.0
74,2,11,6.0
78,1,14,10.0


In [ ]:
y.head()

10    False
25     True
49    False
74    False
78    False
Name: order_banana, dtype: bool

# Build the Model

But first, what's our baseline?

In [ ]:
y.value_counts(normalize=True)

False    0.736779
True     0.263221
Name: order_banana, dtype: float64

Train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, 
                                                  test_size=0.2, 
                                                  random_state=42)

Build the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Instatiate
lr_model = RandomForestClassifier()

# Fit model
lr_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
print('training accuracy', lr_model.score(X_train, y_train))
print('validation accuracy', lr_model.score(X_val, y_val))

training accuracy 0.7403279125820496
validation accuracy 0.7299367426263242


In [ ]:
X.head()

,order_dow,order_hour_of_day,days_since_prior_order
10,4,8,14.0
25,1,11,30.0
49,0,11,6.0
74,2,11,6.0
78,1,14,10.0


In [ ]:
order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


# Confusions

In [ ]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
